In [1]:
import json
import os
import sys
from datetime import datetime
import pandas as pd
import torch
from torch import BoolTensor
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

MODEL_NAME = 'distilbert-base-uncased'
MAX_LENGTH = 512

DIR_OUTPUT = 'results'
DEVICE_DEFAULT = 'cuda'

def get_ts():
    return datetime.utcnow().replace(microsecond=0).isoformat()
# end


class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
    # end
# end

def read_passages(path_data, path_label, test_size=0):
    df = pd.read_csv(path_data)

    documents = df['processed'].to_list()
    labels_str = df['target'].to_list()

    samples = documents

    with open(path_label, 'r') as file:
        labels_list = sorted(json.load(file))
    # end

    labels_all = {l: idx for idx, l in enumerate(labels_list)}

    labels = [labels_all[label_str] for label_str in labels_str]

    if test_size > 0:
        return train_test_split(samples, labels, test_size=test_size, stratify=labels, random_state=234), labels_list
    else:
        return (samples, samples, labels, labels), labels_list
    # end
# end


def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
# end

def predict_plus(input_tokenized, model):

    out = model(**input_tokenized.to(DEVICE_DEFAULT), output_attentions=True)
    logits = out.logits.cpu()
    return logits
# end

def main_train_and_evaluate(name_train, path_train, path_label, path_test, path_output):
    print('[{}] start main_train_and_evaluate with {} {}'.format(get_ts(), path_train, path_test))

    model_name = MODEL_NAME
    max_length = MAX_LENGTH
    output_dir = DIR_OUTPUT

    (train_samples, valid_samples, train_labels, valid_labels), target_names = read_passages(path_train, path_label,
                                                                                             0.1)

    tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
    train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length,
                                                  return_tensors='pt')
    valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length,
                                                  return_tensors='pt')

    train_dataset = SimpleDataset(train_encodings, train_labels)
    valid_dataset = SimpleDataset(valid_encodings, valid_labels)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))

    training_args = TrainingArguments(
        output_dir=output_dir,  # output directory
        num_train_epochs=8,  # total number of training epochs
        per_device_train_batch_size=8,  # batch size per device during training
        per_device_eval_batch_size=8,  # batch size for evaluation
        warmup_steps=0,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        load_best_model_at_end=True,
        # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
        logging_steps=1,  # log & save weights each logging_steps
        evaluation_strategy="epoch",  # evaluate each `logging_steps`
        learning_rate=2e-5,
        save_strategy='epoch',
        save_total_limit=6,
        metric_for_best_model='f1'
    )

    trainer = Trainer(
        model=model,  # the instantiated Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=train_dataset,  # training dataset
        eval_dataset=valid_dataset,  # evaluation dataset
        compute_metrics=compute_metrics,  # the callback that computes metrics of interest
        callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
    )

    # trainer = Trainer(
    #     model=model,  # the instantiated Transformers model to be trained
    #     args=training_args,  # training arguments, defined above
    #     train_dataset=train_dataset,  # training dataset
    #     eval_dataset=valid_dataset,  # evaluation dataset
    #     compute_metrics=compute_metrics
    # )

    print('[{}] start training...'.format(get_ts()))
    trainer.train()

    info_state_model = trainer.evaluate()
    print('[{}] finish training.'.format(get_ts()))

    ################## start to do eval ##################

    (samples_test, _, indexs_label_test, _), target_names = read_passages(path_test, path_label, 0)
    labels_test = [target_names[index_label_test] for index_label_test in indexs_label_test]

    list_conf_output = []
    list_label_output = []
    list_answer_output = []

    for sample_test, label_origin in zip(samples_test, labels_test):
        input_tokenized = tokenizer.encode_plus(sample_test, padding=True, truncation=True, max_length=max_length,
                                                return_tensors='pt')
        with torch.no_grad():
            # out = model(**input_tokenized, output_hidden_states=True, output_attentions=True)
            logits_this = predict_plus(input_tokenized, model)
        # end

        probas_evaluate = torch.nn.functional.softmax(logits_this, dim=-1)
        answer_evaluate = int(probas_evaluate.argmax())
        label_evaluate = target_names[answer_evaluate]

        list_conf_output.append(probas_evaluate.numpy().tolist()[0][answer_evaluate])
        list_label_output.append(label_evaluate)
        list_answer_output.append(label_origin)
    # end

    print('[{}] finish testing.'.format(get_ts()))

    pairs_label_conf = [[a, b, c] for a, b, c in zip(list_label_output, list_conf_output, list_answer_output)]

    filename_output = f'output-{name_train}.json'
    path_file_output = os.path.join(path_output, filename_output)

    with open(path_file_output, 'w+') as file:
        file.write(json.dumps(pairs_label_conf))
    # end

    print('[{}] main_train_and_evaluate finished.'.format(get_ts()))
    

# end


In [2]:
path_folder_train = 'data'
path_label = 'labels.json'
path_output = 'output_noseed'

import numpy as np
import random
import torch

seed_val = 234
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

import os
import subprocess

filenames = sorted([filename for filename in os.listdir(path_folder_train) if filename[0] != '.' and 'train' in filename])

for _filename in filenames:
    filename_base = _filename.split('_')[0]
    
    filename_test = f'{filename_base}_test.csv'
    filename_train = _filename
    
    
    path_train = os.path.join(path_folder_train, filename_train)
    path_test = os.path.join(path_folder_train, filename_test)
    
    main_train_and_evaluate(filename_base, path_train, path_label, path_test, path_output)

    subprocess.run("rm -rf results", shell=True)
    # subprocess.run("rm -rf mlruns", shell=True)
# end

[2023-08-05T13:18:19] start main_train_and_evaluate with data/202304270449_train_0.35_15.csv data/202304270449_test.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

[2023-08-05T13:18:37] start training...


<ipython-input-1-1e70c23b4961>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000300,0.057831,0.987962,0.987977,0.987962,0.987936
2,0.000100,0.048609,0.989556,0.989623,0.989556,0.989551
3,0.000000,0.039825,0.991503,0.991607,0.991503,0.991499
4,0.000000,0.022578,0.995397,0.995406,0.995397,0.995398
5,0.000000,0.026575,0.994335,0.994369,0.994335,0.994333
6,0.000000,0.029201,0.994689,0.994696,0.994689,0.994685
7,0.000000,0.030692,0.995751,0.995753,0.995751,0.995752
8,0.000000,0.039939,0.995043,0.995043,0.995043,0.995041


...

[2023-08-06T07:13:06] start training...


<ipython-input-1-1e70c23b4961>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000300,0.076514,0.984598,0.984766,0.984598,0.984499
2,0.000100,0.056012,0.987270,0.987349,0.987270,0.987226
3,0.000000,0.030446,0.994185,0.994195,0.994185,0.994178
4,0.000100,0.027669,0.993556,0.993568,0.993556,0.993549
5,0.000000,0.028258,0.994342,0.994364,0.994342,0.994334
6,0.000000,0.028954,0.994971,0.994986,0.994971,0.994965
7,0.000000,0.034650,0.994971,0.994986,0.994971,0.994964
8,0.000000,0.039575,0.994814,0.994827,0.994814,0.994808


***** Running Evaluation *****
  Num examples = 6363
  Batch size = 8
Saving model checkpoint to results/checkpoint-7159
Configuration saved in results/checkpoint-7159/config.json
Model weights saved in results/checkpoint-7159/pytorch_model.bin
<ipython-input-1-1e70c23b4961>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 6363
  Batch size = 8
Saving model checkpoint to results/checkpoint-14318
Configuration saved in results/checkpoint-14318/config.json
Model weights saved in results/checkpoint-14318/pytorch_model.bin
<ipython-input-1-1e70c23b4961>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tor

[2023-08-06T09:55:13] finish training.
[2023-08-06T09:55:13] finish testing.
[2023-08-06T09:55:13] main_train_and_evaluate finished.


In [3]:
print('hello')

hello
